In [41]:
!pip install -U openai-whisper

In [42]:
!pip install openai-whisper pydub


In [43]:
import os
import zipfile
from pathlib import Path
import whisper
from pydub import AudioSegment

# Initialize Whisper Model
model = whisper.load_model("base")

# Path to the ZIP file
zip_file_path = "Audio_Song_Actors_01-24 (1).zip"
output_transcripts = []

# Step 1: Extract the ZIP file
extract_dir = "extracted_audio_files"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)




/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [44]:
# Step 2: Process each audio file
for file in Path(extract_dir).rglob('*'):
    if file.suffix.lower() in [".mp3", ".wav", ".m4a"]:  # Supported audio formats
        print(f"Processing: {file.name}")

        # Optional: Convert file to WAV using pydub (if not already in WAV format)
        if file.suffix.lower() != ".wav":
            audio = AudioSegment.from_file(file)
            file = file.with_suffix(".wav")  # Change the file extension to .wav
            audio.export(file, format="wav")

        # Step 3: Run Whisper
        transcription = model.transcribe(str(file))
        output_transcripts.append((file.name, transcription['text']))



Processing: 03-02-06-01-01-01-03.wav
Processing: 03-02-02-01-02-01-03.wav
Processing: 03-02-01-01-01-01-03.wav
Processing: 03-02-06-02-01-02-03.wav
Processing: 03-02-06-02-02-02-03.wav
Processing: 03-02-04-01-01-02-03.wav
Processing: 03-02-01-01-02-01-03.wav
Processing: 03-02-03-02-02-01-03.wav
Processing: 03-02-04-01-01-01-03.wav
Processing: 03-02-06-02-01-01-03.wav
Processing: 03-02-04-01-02-01-03.wav
Processing: 03-02-04-02-02-02-03.wav
Processing: 03-02-03-02-02-02-03.wav
Processing: 03-02-02-01-01-02-03.wav
Processing: 03-02-02-01-01-01-03.wav
Processing: 03-02-04-02-01-01-03.wav
Processing: 03-02-03-01-01-01-03.wav
Processing: 03-02-05-01-02-02-03.wav
Processing: 03-02-03-02-01-01-03.wav
Processing: 03-02-01-01-01-02-03.wav
Processing: 03-02-03-01-02-02-03.wav
Processing: 03-02-03-01-02-01-03.wav
Processing: 03-02-02-01-02-02-03.wav
Processing: 03-02-05-01-01-02-03.wav
Processing: 03-02-02-02-02-02-03.wav
Processing: 03-02-01-01-02-02-03.wav
Processing: 03-02-05-02-02-01-03.wav
P

In [45]:
# Step 4: Save or Print the Transcriptions
output_file = "transcriptions.txt"
with open(output_file, 'w') as f:
    for filename, text in output_transcripts:
        f.write(f"{filename}:\n{text}\n\n")

print(f"Transcriptions saved to {output_file}")

Transcriptions saved to transcriptions.txt


In [46]:
from google.colab import files
files.download('transcriptions.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
from textblob import TextBlob

# File path to your text file
file_path = "transcriptions.txt"

# Dictionary to store results
classifications = {}

# Define a classification function
def classify_tone(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0:
        return "Positive"
    elif polarity < 0:
        return "Negative"
    else:
        return "Neutral"

# Read and classify the text file
with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line and ":" in line:  # Ensure the line contains valid data
            filename, transcription = line.split(":", 1)
            filename = filename.strip()
            transcription = transcription.strip()
            tone = classify_tone(transcription)  # Classify the transcription
            classifications[filename] = {"transcription": transcription, "tone": tone}

# Print the results
for filename, data in classifications.items():
    print(f"Filename: {filename}\nTranscription: {data['transcription']}\nTone: {data['tone']}\n")

# Optional: Save results to a new file
output_file = "classified_transcriptions.txt"
with open(output_file, 'w') as f:
    for filename, data in classifications.items():
        f.write(f"{filename}:\n{data['transcription']} ({data['tone']})\n\n")


Filename: 03-02-06-01-01-01-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-02-01-02-01-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-01-01-01-01-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-06-02-01-02-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-06-02-02-02-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-04-01-01-02-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-01-01-02-01-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-03-02-02-01-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-04-01-01-01-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-06-02-01-01-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-04-01-02-01-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-04-02-02-02-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-03-02-02-02-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-02-01-01-02-03.wav
Transcription: 
Tone: Neutral

Filename: 03-02-02-01-01-01-03.wav
Transcription: 
Tone: Neutral

Filename: 

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
from textblob import TextBlob

# Define function for sentiment classification
def classify_sentiment(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity  # Polarity ranges from -1 (negative) to +1 (positive)
    if polarity > 0:
        return "Positive"
    elif polarity < 0:
        return "Negative"
    else:
        return "Neutral"

# Example transcriptions
transcriptions = {
    "03-02-06-01-01-01-03.wav": "Kids are talking by the door.",
    "03-02-02-01-02-01-03.wav": "Dogs are sitting by the door."
}

# Classify each transcription
for file, text in transcriptions.items():
    sentiment = classify_sentiment(text)
    print(f"File: {file}\nTranscription: {text}\nSentiment: {sentiment}\n")


File: 03-02-06-01-01-01-03.wav
Transcription: Kids are talking by the door.
Sentiment: Neutral

File: 03-02-02-01-02-01-03.wav
Transcription: Dogs are sitting by the door.
Sentiment: Neutral



In [50]:
from transformers import pipeline

# Load pre-trained sentiment analysis model
sentiment_analyzer = pipeline("sentiment-analysis")

# Example transcriptions
transcriptions = [
    "Kids are talking by the door.",
    "Dogs are sitting by the door."
]

# Analyze sentiment
for text in transcriptions:
    result = sentiment_analyzer(text)[0]
    print(f"Text: {text}\nSentiment: {result['label']} (Score: {result['score']:.2f})\n")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Text: Kids are talking by the door.
Sentiment: POSITIVE (Score: 0.98)

Text: Dogs are sitting by the door.
Sentiment: NEGATIVE (Score: 0.95)

